In [2]:
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn

In [3]:
# Preprocessing 
df = pd.read_csv("datasets/insurance.csv")
df = df.drop(columns=['region'])
df['sex']=df['sex'].map({'female':0,'male':1}).astype(int)
df['smoker']=df['smoker'].map({'yes':0,'no':1}).astype(int)

In [4]:
# Training Data
X = df[["age", "bmi", "children", "smoker", "charges"]]
y = df[["sex"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
# Eval Data
eval_data = X_test
eval_data["sex"] = y_test

In [6]:
# Model
lr_class = LogisticRegression().fit(X_train, y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Log the model using Scikit-Learn Flavor
mlflow.sklearn.log_model(lr_class, "model")

C:\ProgramData\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/07 19:10:20 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [8]:
# Get run id
run = mlflow.last_active_run()
run_id = run.info.run_id
run_id

'aa1a55e6db784f8e897de8aba06a47f9'

In [9]:
mlflow.evaluate(f"runs:/{run_id}/model", 
                data=eval_data, 
                targets="sex",
				model_type="classifier"
)

C:\ProgramData\anaconda3\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/08/07 19:10:22 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/08/07 19:10:22 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, neg